In [29]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node ad6b0acc-2bf0-4a27-bb7a-64d9d8bda21d

In [30]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
from tqdm import tqdm
import time

In [31]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [32]:
#Thymio control
def motors(l_speed=500, r_speed=500):
    return {
        "motor.left.target": [l_speed],
        "motor.right.target": [r_speed],
    }

async def forward(motor_speed):
    await node.set_variables(motors(motor_speed,motor_speed))

async def rotate(theta, motor_speed): #theta is in radians
    direction_rot=(theta>=0)-(theta<0)
    await node.set_variables(motors(motor_speed*direction_rot, -motor_speed*direction_rot))
    # wait time to get theta 1.44 is the factor to correct
    time=(theta)*100/motor_speed*1.44
    await(client.sleep(time))
    # stop the robot
    await node.set_variables(motors(0, 0))

async def stop_motor():
    await node.set_variables(motors(0,0))

async def get_proximity_values():
    # Wait for the Thymio node
    node = await client.wait_for_node()
    # Wait for the proximity sensor variables
    await node.wait_for_variables({"prox.horizontal"})
    # Get the proximity values : v: Stands for "variables" and is used to access the cached variable values.
    proximity_values = node.v.prox.horizontal
    # Return the value of the front proximity sensor (index 2)
    return proximity_values[0:5]

# Run the asynchronous function and print the result
proximity_values = await get_proximity_values()
print(f"Front Proximity Sensor Value: {proximity_values}")

Front Proximity Sensor Value: [0, 0, 0, 0, 0]


In [33]:
## Parameters for local navigation
threshold_obst = 3500 
threshold_loc = 2500
local_motor_speed = 100
threshold_obst_list = [3200, 3600, 3600, 3600, 3200]

async def local_navigation():
    threshold_obst = 1000
    threshold_loc = 800
    local_motor_speed = 100
    threshold_obst_list = [3200, 3600, 3600, 3600, 3200]
    sens = await get_proximity_values()

    # Follow the obstacle by the left
    if (sens[0] + sens[1]) > (sens[4] + sens[3]):
        await bypass('right', sens, threshold_loc, local_motor_speed)

    # Follow the obstacle by the right    
    else:
        await bypass('left', sens, threshold_loc, local_motor_speed)

async def bypass(leftright, sens, threshold_loc, local_motor_speed):
    if leftright == "right":
        while sum(sens[i] > threshold_obst for i in range(0, 5)) > 0:
            print("Turn right")
            await rotate(np.pi/6/2, local_motor_speed)
            #await asyncio.sleep(0.2)
            sens = await get_proximity_values()
            print(sens)

        while sens[0] > threshold_loc:
            await forward(local_motor_speed)
            #await asyncio.sleep(0.2)
            #time.sleep(0.2)
            sens = await get_proximity_values()
        
        

    elif leftright == "left":
        while sum(sens[i] > threshold_loc for i in range(0, 5)) > 0:
            print("Turn left")
            await rotate(-np.pi/6, local_motor_speed)
            #await asyncio.sleep(0.2)
            sens = await get_proximity_values()

        while sens[4] > threshold_loc:
            await forward(local_motor_speed)
            #await asyncio.sleep(0.2)
            #time.sleep(0.2)
            sens = await get_proximity_values()

    if(leftright=="right" and sens[0] > threshold_loc):
        await rotate(np.pi/2, 100)
    

    await forward(local_motor_speed)
    #time.sleep(2)
    await stop_motor()

In [34]:
thymio_data = []
#acquire_data = True
Ts = 0.1

def get_data():
    thymio_data.append({"sensor":node["prox.horizontal"],
                        "left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})

async def get_speed():
    await node.wait_for_variables() # wait for Thymio variables values
    rt = RepeatedTimer(Ts, get_data) # it auto-starts, no need of rt.start()

    try:
        await client.sleep(Ts)
    finally:
        rt.stop() # better in a try/finally block to make sure the program ends!
        node.send_set_variables(motors(0, 0))

def pos_estimation(left_speed, right_speed, dt, pos0_x, pos0_y, theta0):
    real_thymio_speed = 21.739 #mm/s
    #speed estimation
    speed_measured = (right_speed[-1] + left_speed[-1])/2
    speed = (speed_measured * 50) / real_thymio_speed 
    angular_speed_measured = (right_speed[-1] - left_speed[-1])/2
    angular_speed = (angular_speed_measured * 50) / real_thymio_speed

    #position estimation
    theta = theta0 + angular_speed * dt
    pos_x = pos0_x + speed * np.cos(theta) * dt
    pos_y = pos0_y + speed * np.sin(theta) * dt

    return speed, pos_x, pos_y, theta

In [35]:
#MAIN CODE : 

#parameters
path_step = 0 #This is the index of where we are in the path.
path_coord = np.array([[1,2], [2,3], [3,4]])
b = np.array([3.2, 4.5])
epsilon = 0.5
local_obstacle = False
threshold_obst = 3500 #threshold for obstacle detection
pos_x, pos_y, theta = 0, 0, 0
speed_history = []
thymio_data.clear()

#to implement in a different way thanks to filtering : 
robot_position = 1
end_position = 2

async def main():

    global local_obstacle, pos_x, pos_y, theta, dt
    while(robot_position != end_position):
        start_time = time.time()
        sens = await get_proximity_values()
        if (sum(sens[i] > threshold_obst for i in range(0, 5)) > 0):
            local_obstacle = True

        if(local_obstacle):
            await local_navigation()
            start_time = 0          
        
        #if(find_index_path_if_close(path_coord,b,epsilon) and time.time-start_time>2): 
            #path_step=find_index_path_if_close(path_coord, position_robot, epsilon)
            #local_navigation=False

        if not local_obstacle: 
            node.send_set_variables(motors(50, 50))
            await get_speed()
            #if len(thymio_data) == 0: return pos0_x, pos0_y, theta0
    
            #current variables
            left_speed = [x["left_speed"] for x in thymio_data]
            right_speed = [x["right_speed"] for x in thymio_data]
            dt = time.time() - start_time

            speed, pos_x, pos_y, theta = pos_estimation(left_speed, right_speed, dt, pos_x, pos_y, theta)
            speed_history.append(speed)
            print(round(pos_x,2), round(pos_y,2), round(theta,2))

await main()
var_speed = np.var(speed_history)
std_speed = np.std(speed_history)

print("The speed variance in mm^2/s^2 is {}".format(std_speed))

0.42 -1.15 -1.22
0.41 -1.51 -1.58
-0.48 -8.42 -1.7
-12.04 -14.05 -2.69
-22.66 -17.72 -2.81
-33.77 -20.09 -2.93
-43.57 -20.96 -3.05
-53.74 -20.59 -3.18
-63.77 -18.95 -3.3
-73.69 -15.99 -3.43
-83.02 -18.0 -2.93
-90.19 -24.14 -2.43
-76.47 -23.86 -6.26
-68.33 -30.94 -7.0
-57.15 -30.96 -6.28
-61.91 -20.78 -4.27
-53.12 -23.93 -0.34
-43.46 -28.88 -0.47
-34.92 -34.71 -0.6
-31.05 -45.39 -1.22
-37.18 -56.73 -2.07
-36.6 -54.18 -4.94
-35.6 -44.08 -4.81
-32.9 -59.89 -1.4
-26.75 -70.49 -1.04
-23.7 -80.85 -1.28
-31.48 -71.01 -4.04
-41.84 -65.12 -3.66
-53.33 -63.47 -3.28
-64.36 -71.01 -2.54
-58.75 -81.13 -1.07
-48.83 -85.47 -0.41
-45.58 -95.44 -1.26
-48.35 -109.45 -1.77
-56.68 -119.91 -2.24
-57.18 -131.35 -1.61
-45.67 -137.35 -0.48
-35.4 -129.77 0.64
-22.88 -129.43 0.03
-11.96 -130.45 -0.09
-16.14 -120.06 1.95
-25.08 -114.26 2.57
-12.48 -108.28 6.73
-5.58 -98.76 7.23
-4.29 -89.61 7.71
6.64 -80.65 6.97
20.61 -81.34 6.23
31.42 -74.6 6.84
36.61 -62.16 7.46
33.89 -50.4 8.08
45.83 -57.39 5.75
32.58 -61.32 

CancelledError: 

In [17]:
await node.set_variables(motors(0, 0))


{'error_code': 2}

In [ ]:
var_speed = np.var(speed/thymio_speed_to_mms)
std_speed = np.std(speed/thymio_speed_to_mms)

print("The speed variance in mm^2/s^2 is {}".format(std_speed))

In [ ]:
q_nu = std_speed / 2 # variance on speed state
r_nu = std_speed / 2 # variance on speed measurement 

In [88]:
pos_x, pos_y, theta = 0, 0, 0
thymio_data.clear()

while True:
    #await client.sleep(5)
    node.send_set_variables(motors(50, 50))
    await get_speed()
    pos_x, pos_y, theta = pos_estimation(thymio_data, Ts, pos_x, pos_y, theta)
    print(pos_x, pos_y, theta)
    prox_sensor = [x["sensor"][2] for x in thymio_data]   
    if (len(prox_sensor)!=0) and (prox_sensor[-1]>threshold):
        acquire_data=False
        break

50.0 0.0
92.56275425168309 23.252138622303846
143.0627542516831 23.252138622303846
170.61817185095822 66.16715884750657
149.39468318705394 112.54132761561634
129.00348819624395 157.09690153007475
109.02844004198111 200.74317801770746
89.05339188771828 244.38945450534018
68.24605006036116 289.85432584662425


In [ ]:
await tdmclient.notebook.stop()

ValueError: Function <function _pre_run_cell at 0x7f8be4798d30> is not registered as a pre_run_cell callback